In [2]:
##-----Imports-----##
from Base import *

In [3]:
##-----Constants-----##
MASSTIMECOEFF = 0.03
MAXWEIGHT = 100
YEAR = 2021
FILE_NAME = '2021 Season.xlsx'
FASTEST_TEAM = 'Mercedes'

##-----Lists & Dictionaries-----##
event_dict = {}
baseline_dict = {}
event_list = []

##-----Dataframes-----##
fullregression_df = pd.DataFrame(columns=['Driver', 'Stint', 'Compound', 'Slope', 'Intercept', 'R Value', 'Standard Error', 'No of Laps', 'Laps Till End', 'Team', 'Grand Prix'])
regression_df = pd.DataFrame(columns=['Compound', 'Slope', 'Intercept', 'R Value', 'Standard Error', 'No of Laps', 'Laps Till End', 'Team', 'Average Lap Time'])
avgdeg_df = pd.DataFrame(columns = ['Team', 'Compound', 'Average Degradation', 'No of Laps','Average Lap Time', 'Delta Lap Time'])
eventdeg_df = pd.DataFrame(columns = ['Team', 'Event', 'Event Number', 'Average Degradation', 'No of Laps', 'Average Lap Time'])
ranking_df = pd.DataFrame(columns = ['Team', 'Event', 'Event Number', 'Degradation Rank', 'Lap Time Rank'])
rankingchange_df = pd.DataFrame(columns = ['Degradation Change', 'Lap Time Change'])
difference_df = pd.DataFrame(columns = ['Team', 'Event', 'Event Number', 'Delta Degradation', 'Delta Laptime'])
differencechange_df = pd.DataFrame(columns = ['Degradation Change', 'Lap Time Change'])


##-----Variables-----##
number = 1
lap_time_value = 0
new_value = 0
std_error_dev = 0
std_error_mean = 0
std_error_min = 0
std_error_max = 0
slope_dev = 0
slope_mean = 0
slope_min = 0
slope_max = 0
no_of_laps = 0
laps_till_end = 0
team = ''
compound = ''
avg_laptime = 0
slope = 0
intercept = 0
r_value = 0
p_value = 0
std_err = 0
avg_deg = 0
compound_laps = 0
delta_laptime = 0
deg_slope = 0
time_slope = 0
event_number = 0
min_deg = 0
min_lap = 0
delta_deg = 0

In [17]:
##-----Functions-----##
def getLaps(fyear, fevent):
    session = ff1.get_event(fyear, fevent).get_race()
    session.load()
    laps = session.laps
    laps = laps.pick_accurate().pick_track_status('1')
    return laps

def fuelDelta(flaps_df):
    max_laps = flaps_df['LapNumber'].max()
    delta_fuel_mass = MAXWEIGHT / max_laps #Kgs of fuel lost for each lap
    fuel_delta = MASSTIMECOEFF * delta_fuel_mass #Coefficient of weight vs time x kgs per lap
    return (max_laps, fuel_delta)

def groupLaps(fdataframe, fname):
    groupedDataframe = fdataframe.groupby(fname)
    return groupedDataframe

def baseLapTime(fdataframe):
    dict = {}

    baseline_df = fdataframe[fdataframe['Team'] == FASTEST_TEAM] #Filter dataframe so that only Red Bull Racing is used
    groupedDataframe = groupLaps(baseline_df, 'Compound') #Group baseline_df by the Compound

    for compound_name, compound_df in groupedDataframe:
        dict[compound_name] = compound_df['Average Lap Time'].mean() #For each compound calculate the mean lap time and save it to a dictionary

    return dict

In [18]:
##-----Get Event List-----##
event_list = ff1.get_event_schedule(YEAR, include_testing=False)[['EventName', 'RoundNumber']]
event_dict = event_list.set_index('EventName').to_dict()['RoundNumber'] #Creates a dictionary which can reference the Event Name vs the Round Number
event_list = ff1.get_event_schedule(YEAR, include_testing=False)['EventName'].to_list()

In [ ]:
##-----Take All Data from Year & Convert into Excel Base-----##
with pd.ExcelWriter(FILE_NAME) as writer:
    for event in event_list:
        ##Load & Initalise laps for each event##
        laps = getLaps(YEAR, event)

        ##Calculate the time loss in seconds per lap due to fuel##
        max_laps, fuel_delta = fuelDelta(laps)


        for row in laps.itertuples(index = True):
            index = row.Index

            lap_time_value = row.LapTime

            ##Change in time per lap times the number of laps passed##
            new_value = lap_time_value + pd.Timedelta(seconds = (fuel_delta * (row.LapNumber - 1)))

            laps.at[index, 'FuelAdjustedLapTime'] = new_value #Assign new value


        groupedLaps = groupLaps(laps, 'Driver') #Group laps by driver

        for driver_name, driver_df in groupedLaps:

            groupedStints = groupLaps(driver_df, 'Stint') #Group driver laps by stint number

            for stint_name, stint_df in groupedStints:

                stint_df['FuelAdjustedLapTime'] = stint_df['FuelAdjustedLapTime'].dt.total_seconds() #Convert laptime from TimeDelta to seconds

                ##Calculate number of laps in the stint, number of laps till end, team, compound used, average laptime##
                no_of_laps = len(stint_df)
                laps_till_end = max_laps - stint_df['LapNumber'].min()
                team = stint_df['Team'].iloc[0]
                compound = stint_df['Compound'].iloc[0]
                avg_laptime = stint_df['FuelAdjustedLapTime'].mean()

                slope, intercept, r_value, p_value, std_err = linregress(stint_df['LapNumber'], stint_df['FuelAdjustedLapTime'])

                regression_df.loc[f'{driver_name} -  Stint {stint_name}'] = [compound, slope, intercept, r_value, std_err, no_of_laps, laps_till_end, team, avg_laptime]
                    #Add new row to the regression dataframe for each event

                row_df = pd.DataFrame([{'Driver': driver_name, 'Stint': stint_name, 'Compound': compound, 'Slope': slope, 'Intercept': intercept, 'R Value': r_value, 'Standard Error': std_err, 'No of Laps': no_of_laps, 'Laps Till End': laps_till_end, 'Team': team, 'Grand Prix': event, 'Average Lap Time': avg_laptime}])
                    #Can't use simpler .loc[] because the key repeats multiple times - e.g. ALB - Stint 1 can happen multiple times in multiple races

                fullregression_df = pd.concat([fullregression_df, row_df], ignore_index=True)
                    #Add new row to the regression dataframe for all events

        regression_df.to_excel(writer, sheet_name=event)
    fullregression_df.to_excel(writer, sheet_name='Full Season', index = False)

master_fullregression_df = fullregression_df #Save to a master copy so that any changes made by mistake can be reverted

In [20]:
fullregression_df = master_fullregression_df #Code to access master copy in case of mistake

In [21]:
##-----Filter Data-----##
fullregression_df = fullregression_df[(fullregression_df['No of Laps'] > 5) & (fullregression_df['Laps Till End'] > 9) & (fullregression_df['Compound'] != 'INTERMEDIATE') & (fullregression_df['Compound'] != 'WET')]
    #Filters the full regression so that Number of Laps > 5; Laps Till End > 9 and the Compound is not Wet or Intermediate

##Calculate two Standard Deviations of the 'Standard Error' Column##
std_error_dev = fullregression_df['Standard Error'].std()
std_error_mean = fullregression_df['Standard Error'].mean()
std_error_min = std_error_mean - 2 * std_error_dev
std_error_max = std_error_mean + 2 * std_error_dev

##Calculate two Standard Deviations of the 'Slope' Column##
slope_dev = fullregression_df['Slope'].std()
slope_mean = fullregression_df['Slope'].mean()
slope_min = slope_mean - 2 * slope_dev
slope_max = slope_mean + 2 * slope_dev


fullregression_df = fullregression_df[(fullregression_df['Standard Error'] >= std_error_min) & (fullregression_df['Standard Error'] <= std_error_max) & (fullregression_df['Slope'] >= slope_min) & (fullregression_df['Slope'] <= slope_max)]
    #Filters the full regression further so that all values lie within two standard deviations on a normal distribution

filteredregression_df = fullregression_df #Save to an editable copy so that any changes are not done to the master sheet

##Write into a new sheet in the Excel Workbook##
with pd.ExcelWriter(FILE_NAME,engine='openpyxl', mode = 'a') as writer:
    filteredregression_df.to_excel(writer, sheet_name='Filtered Full Season', index = False)

In [22]:
##-----Compare Degradation for each Compound-----##
groupedLaps = groupLaps(filteredregression_df, 'Team') #Group the filtered regression by Team
baseline_dict = baseLapTime(filteredregression_df) #Calculate the mean lap time for each tyre by the fastest team (i.e. Red Bull Racing)

with pd.ExcelWriter(FILE_NAME,engine='openpyxl', mode = 'a') as writer:
    for team_name, team_df in groupedLaps:

        groupedCompounds = groupLaps(team_df, 'Compound') #Group each teams regression by Compound

        for compound_name, compound_df in groupedCompounds:

            ##Calculate average degradation; number of laps on the compound; average lap time; difference to the fastest teams lap time
            avg_deg = compound_df['Slope'].mean()
            compound_laps = compound_df['No of Laps'].sum()
            avg_laptime = compound_df['Average Lap Time'].mean()
            delta_laptime = avg_laptime - baseline_dict[compound_name]

            avgdeg_df.loc[number] = [team_name, compound_name, avg_deg, compound_laps, avg_laptime, delta_laptime]

            number += 1
    avgdeg_df.to_excel(writer, sheet_name='Degradation Comparison', index = False)

In [23]:
##-----Compare average degradation for each team for each event-----##
groupedLaps = groupLaps(filteredregression_df, 'Grand Prix') #Group filtered regression by the Grand Prix

with pd.ExcelWriter(FILE_NAME,engine='openpyxl', mode = 'a') as writer:
    for event_name, event_df in groupedLaps:

        groupedTeams = groupLaps(event_df, 'Team') #Group event regression by Team

        for team_name, team_df in groupedTeams:

            ##Calculate average degradation, number of laps run, average laptime at the event##
            avg_deg = team_df['Slope'].mean()
            no_of_laps = team_df['No of Laps'].sum()
            avg_laptime = team_df['Average Lap Time'].mean()
            event_number = event_dict[event_name]

            eventdeg_df.loc[number] = [team_name, event_name, event_number, avg_deg, no_of_laps, avg_laptime]

            number += 1

    eventdeg_df.to_excel(writer, sheet_name='Event Degradation Comparison', index = False)

In [24]:
##-----Calculate a ranking for each team across each event-----##
groupedEvents = groupLaps(eventdeg_df, 'Event') #Group event degradation by Event

with pd.ExcelWriter(FILE_NAME, engine='openpyxl', mode = 'a') as writer:
    for event_name, event_df in groupedEvents:

        ##For each event, rank the average degradation and the average lap time##
        event_df['Degradation Rank'] = event_df['Average Degradation'].rank()
        event_df['Lap Time Rank'] = event_df['Average Lap Time'].rank()
        event_df['Event Number'] = event_dict[event_name]

        ranking_df = pd.concat([ranking_df, event_df[['Team', 'Event', 'Event Number', 'Degradation Rank', 'Lap Time Rank']]])

    ranking_df.to_excel(writer, sheet_name='Team Ranking', index = False)

In [25]:
##-----Calculate change and movement by teams across the year in regard to degradation and lap time-----##
##Convert values into integers from floats to be used by linregress function##
ranking_df['Event Number'] = pd.to_numeric(ranking_df['Event Number'], errors='coerce')
ranking_df['Degradation Rank'] = pd.to_numeric(ranking_df['Degradation Rank'], errors='coerce')
ranking_df['Lap Time Rank'] = pd.to_numeric(ranking_df['Lap Time Rank'], errors='coerce')


groupedTeams = groupLaps(ranking_df, 'Team') #Group ranked teams by Team, to get the ranking for each event across the year

with pd.ExcelWriter(FILE_NAME, engine='openpyxl', mode = 'a') as writer:
    for team_name, team_df in groupedTeams:

        ##Calculate slope of Degradation & Lap Time Rank across the Events.##
        slope, _, _, _, _  = linregress(team_df['Event Number'], team_df['Degradation Rank']) #_ is used because there is no need for the other values
        deg_slope = slope

        slope, _, _, _, _  = linregress(team_df['Event Number'], team_df['Lap Time Rank']) #_ is used because there is no need for the other values
        time_slope = slope

        rankingchange_df.loc[team_name] = [deg_slope, time_slope]
    rankingchange_df.to_excel(writer, sheet_name ='Team Ranking Change')

In [38]:
##-----Calculate the difference in seconds to the best lap time and degradation for each team for each event-----##
groupedEvents = groupLaps(eventdeg_df, 'Event') #Group the degradation and lap time by event

with pd.ExcelWriter(FILE_NAME, engine='openpyxl', mode='a') as writer:
    for event_name, event_df in groupedEvents:

        ##Calculate the best degradation and lap time for the race##
        min_deg = event_df['Average Degradation'].min()
        min_lap = event_df['Average Lap Time'].min()

        groupedTeams = groupLaps(event_df, 'Team') #Group the data by Team

        for team_name, team_df in groupedTeams:

            ##Find the delta to the min_deg & min_lap##
            delta_deg = team_df['Average Degradation'].iloc[0] - min_deg
            delta_laptime = team_df['Average Lap Time'].iloc[0] - min_lap

            event_number = event_dict[event_name]

            difference_df.loc[number] = [team_name, event_name, event_number, delta_deg, delta_laptime]

            number += 1

    difference_df.to_excel(writer, sheet_name='Difference to Best', index=False)

In [41]:
##-----Find the change in delta_deg and delta_lap for each team across the year-----##
groupedTeams = groupLaps(difference_df, 'Team') #Group data by Team, to get the numbers for each event across the year

with pd.ExcelWriter(FILE_NAME, engine='openpyxl', mode = 'a') as writer:
    for team_name, team_df in groupedTeams:

        team_df = team_df.sort_values(by='Event Number', ascending=True) #Sort the event so the graph is in the right order

        ##Calculate slope of Degradation & Lap Time Difference across the Events##
        slope, _, _, _, _  = linregress(team_df['Event Number'], team_df['Delta Degradation']) #_ is used because there is no need for the other values
        deg_slope = slope

        slope, _, _, _, _  = linregress(team_df['Event Number'], team_df['Delta Laptime']) #_ is used because there is no need for the other values
        time_slope = slope

        differencechange_df.loc[team_name] = [deg_slope, time_slope]

    differencechange_df.to_excel(writer, sheet_name ='Team Difference Change')